In [1]:
import sys
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# 현재 노트북 파일의 경로를 기준으로 프로젝트 루트 경로를 계산
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..'))

# sys.path에 프로젝트 루트 경로가 없으면 추가
if project_root not in sys.path:
  sys.path.append(project_root)


In [2]:
from src.data.db_handler import DBHandler

db_handler = DBHandler(db_name="data_warehouse")

origin_df = db_handler.fetch_data(table_name="stock_master_final")

In [3]:
origin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1208825 entries, 0 to 1208824
Columns: 156 entries, ticker to prdy_vol
dtypes: datetime64[ns](8), float64(61), int64(63), object(24)
memory usage: 1.4+ GB


In [4]:
origin_df

,ticker,prdy_vrss_x,prdy_vrss_sign_x,acml_vol_x,acml_tr_pbmn_x,stck_oprc_x,stck_hgpr_x,stck_lwpr_x,stck_bsop_date,stck_clpr,flng_cls_code,prtt_rate,sale_account,sale_cost,sale_totl_prfi,bsop_prti,op_prfi,thtr_ntin,cras,fxas,total_aset,flow_lblt,fix_lblt,total_lblt,total_cptl,grs,bsop_prfi_inrt,ntin_inrt,roe_val,eps_x,sps,bps_x,rsrv_rate,lblt_rate,bram_depn,crnt_rate,quck_rate,equt_inrt,totl_aset_inrt,cptl_ntin_rate,sale_ntin_rate,sale_totl_rate,payout_rate,ebitda,lstg_stqt,cpta,papr,scts_mket_lstg_dt,prdt_name,prdt_abrv_name,std_pdno,sbst_pric,thdt_clpr,bfdy_clpr,std_idst_clsf_cd_name,idx_bztp_lcls_cd_name,idx_bztp_scls_cd_name,issu_istt_cd,iscd_stat_cls_code,marg_rate,new_hgpr_lwpr_cls_code,bstp_kor_isnm,prdy_vrss_y,prdy_vrss_sign_y,prdy_ctrt,acml_tr_pbmn_y,acml_vol_y,prdy_vrss_vol_rate,stck_oprc_y,stck_hgpr_y,stck_lwpr_y,stck_mxpr,stck_llam,stck_sdpr,wghn_avrg_stck_prc,hts_frgn_ehrt,frgn_ntby_qty,pgtr_ntby_qty,pvt_scnd_dmrs_prc,pvt_frst_dmrs_prc,pvt_pont_val,pvt_frst_dmsp_prc,pvt_scnd_dmsp_prc,dmrs_val,dmsp_val,stck_sspr,aspr_unit,hts_avls,per,pbr,vol_tnrt,eps_y,bps_y,d250_hgpr,d250_hgpr_date,d250_hgpr_vrss_prpr_rate,d250_lwpr,d250_lwpr_date,d250_lwpr_vrss_prpr_rate,stck_dryy_hgpr,dryy_hgpr_vrss_prpr_rate,dryy_hgpr_date,stck_dryy_lwpr,dryy_lwpr_vrss_prpr_rate,dryy_lwpr_date,w52_hgpr,w52_hgpr_vrss_prpr_ctrt,w52_hgpr_date,w52_lwpr,w52_lwpr_vrss_prpr_ctrt,w52_lwpr_date,whol_loan_rmnd_rate,stck_shrn_iscd,cpfn_cnnm,frgn_hldn_qty,last_ssts_cntg_qty,aspr_acpt_hour,askp1,askp10,bidp1,bidp10,askp_rsqn1,askp_rsqn2,askp_rsqn3,askp_rsqn4,askp_rsqn5,askp_rsqn6,askp_rsqn7,askp_rsqn8,askp_rsqn9,askp_rsqn10,bidp_rsqn1,bidp_rsqn2,bidp_rsqn3,bidp_rsqn4,bidp_rsqn5,bidp_rsqn6,bidp_rsqn7,bidp_rsqn8,bidp_rsqn9,bidp_rsqn10,total_askp_rsqn,total_bidp_rsqn,ntby_aspr_rsqn,new_mkop_cls_code,prdy_clpr_vrss_oprc_rate,lwpr_vrss_prpr,lwpr_vrss_prpr_sign,prdy_clpr_vrss_lwpr_rate,hgpr_vrss_prpr,hgpr_vrss_prpr_sign,prdy_clpr_vrss_hgpr_rate,oprc_vrss_prpr,oprc_vrss_prpr_sign,bstp_cls_code,prdy_vol
0,000080,NaN,3,NaN,NaN,6403.0,6403.0,6403.0,1985-01-25,6403.0,00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70133611,363168055000,5000,2009-10-19,하이트진로보통주,하이트진로,KR7000080002,15320,19160,19160,알콜음료 제조업,시가총액규모대,음식료품,00008,55,40.0,NaN,음식료·담배,-20.0,5,-0.10,3151539425,164607,85.87,19170,19180,19110,24900,13430,19180,19144.18,7.58,-17687.0,-23952,19413,19296,19223,19106,19033,19260,19070,15340,10,13438,14.25,1.13,0.23,1345.0,16889,22200,2025-07-07,-13.69,18680,2025-02-04,2.57,22200,-13.69,2025-07-07,18680,2.57,2025-02-04,22200,-13.69,2025-07-07,18680,2.57,2025-02-04,1.07,000080,"3,631 억",5313280,48024,160000,19170,19260,19160,19070,2593,1470,254,181,25,135,3,1174,23,46,298,5086,1207,5708,5010,7074,5596,2932,5652,1119,5904,39682,33778,31,-0.05,50.0,2,-0.36,-20.0,5,0.00,-10.0,5,000080,191693
1,000080,NaN,3,NaN,NaN,6403.0,6403.0,6403.0,1985-01-26,6403.0,00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70133611,363168055000,5000,2009-10-19,하이트진로보통주,하이트진로,KR7000080002,15320,19160,19160,알콜음료 제조업,시가총액규모대,음식료품,00008,55,40.0,NaN,음식료·담배,-20.0,5,-0.10,3151539425,164607,85.87,19170,19180,19110,24900,13430,19180,19144.18,7.58,-17687.0,-23952,19413,19296,19223,19106,19033,19260,19070,15340,10,13438,14.25,1.13,0.23,1345.0,16889,22200,2025-07-07,-13.69,18680,2025-02-04,2.57,22200,-13.69,2025-07-07,18680,2.57,2025-02-04,22200,-13.69,2025-07-07,18680,2.57,2025-02-04,1.07,000080,"3,631 억",5313280,48024,160000,19170,19260,19160,19070,2593,1470,254,181,25,135,3,1174,23,46,298,5086,1207,5708,5010,7074,5596,2932,5652,1119,5904,39682,33778,31,-0.05,50.0,2,-0.36,-20.0,5,0.00,-10.0,5,000080,191693
2,000080,NaN,3,NaN,NaN,6403.0,6403.0,6403.0,1985-01-28,6403.0,00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70133611,363168055000,5000

In [5]:
df = origin_df.copy()

df = df.set_index(['ticker', 'stck_bsop_date']).reset_index()

In [6]:
cols_to_convert = ['bps_x', 'sps', 'eps_x']

for col in cols_to_convert:
  df[col] = pd.to_numeric(df[col], errors='coerce')

In [7]:
import pandas as pd
import numpy as np

def create_data_info_df(dataframe):
  """
  데이터프레임의 각 컬럼에 대한 종합적인 정보(타입, 결측치, 고유값)를
  담은 새로운 데이터프레임을 생성합니다.

  Args:
    dataframe (pd.DataFrame): 분석할 원본 데이터프레임

  Returns:
    pd.DataFrame: 각 컬럼의 정보가 요약된 데이터프레임
  """
  total_rows = len(dataframe)

  # 결측치 정보 계산
  missing_counts = dataframe.isnull().sum()
  missing_percent = ((missing_counts / total_rows) * 100).round(2)

  # 고유값 정보 계산
  nunique_counts = dataframe.nunique()
  nunique_percent = ((nunique_counts / total_rows) * 100).round(2)

  # 데이터 타입 정보
  dtypes = dataframe.dtypes

  # 정보 취합
  info_df = pd.DataFrame({
    'dtype': dtypes,
    'missing_count': missing_counts,
    'missing_percent': missing_percent,
    'nunique_count': nunique_counts,
    'nunique_percent': nunique_percent
  })

  # 결측치 비율이 높은 순으로 정렬
  info_df = info_df.sort_values(by='missing_percent', ascending=False)

  return info_df

# 함수 실행 및 결과 확인
df = df.replace(r'^\s*$', np.nan, regex=True)
df = df.replace(['0', 0], np.nan)
info_df = create_data_info_df(df)

In [8]:
info_df

,dtype,missing_count,missing_percent,nunique_count,nunique_percent
prtt_rate,float64,1207889,99.92,682,0.06
ntin_inrt,float64,229828,19.01,4044,0.33
idx_bztp_scls_cd_name,object,201530,16.67,18,0.00
bsop_prfi_inrt,float64,170344,14.09,4203,0.35
payout_rate,float64,157104,13.00,1297,0.11
...,...,...,...,...,...
stck_bsop_date,datetime64[ns],0,0.00,10021,0.83
pgtr_ntby_qty,int64,0,0.00,200,0.02
hts_frgn_ehrt,float64,0,0.00,197,0.02
wghn_avrg_stck_prc,float64,0,0.00,200,0.02


In [9]:
def evaluate_threshold_impact(info_df, scenarios):
  """
  (지표, 연산자, 값) 튜플 리스트로 정의된 시나리오를 분석하여
  상세 보고서 DataFrame을 생성합니다.

  Args:
    info_df (pd.DataFrame): 분석할 정보 데이터프레임
    scenarios (list): ('지표', '연산자', '값') 형태의 튜플 리스트
  
  Returns:
    pd.DataFrame: 각 시나리오별 분석 결과가 담긴 보고서 데이터프레임
  """
  total_columns = len(info_df)
  report_data = []

  print(f"--- 임계값 영향 분석 (전체 컬럼 수: {total_columns}개) ---")

  # ✨ 수정된 부분: 튜플을 직접 받아 처리
  for metric, operator, value in scenarios:
    if metric not in info_df.columns:
      print(f"'{metric}'은(는) 유효한 컬럼이 아닙니다. 해당 시나리오를 건너뜁니다.")
      continue

    # 조건에 맞는 컬럼 필터링
    condition = f"`{metric}` {operator} {value}"
    filtered_cols = info_df.query(condition)
    
    # 결과 수집
    count = len(filtered_cols)
    percent = round((count / total_columns) * 100, 2)
    feature_names = filtered_cols.index.tolist()
    
    # 보고서에 추가할 행 생성
    report_data.append({
      '시나리오': condition, # 조건 자체를 시나리오 설명으로 사용
      '대상 컬럼 수': count,
      '비율 (%)': percent,
      '해당 컬럼': feature_names
    })
  
  # 최종 보고서 DataFrame 생성
  report_df = pd.DataFrame(report_data)
  return report_df

# --- ✨ 실험: 훨씬 간결해진 시나리오 리스트 ---
scenarios = [
  ('missing_percent', '>', 90),
  ('missing_percent', '>', 30),
  ('nunique_count', '<=', 1),
  ('nunique_count', '==', 0)
]

# 함수 실행 및 보고서 결과 확인
impact_report = evaluate_threshold_impact(info_df, scenarios)
impact_report

--- 임계값 영향 분석 (전체 컬럼 수: 156개) ---


,시나리오,대상 컬럼 수,비율 (%),해당 컬럼
0,`missing_percent` > 90,1,0.64,[prtt_rate]
1,`missing_percent` > 30,1,0.64,[prtt_rate]
2,`nunique_count` <= 1,0,0.00,[]
3,`nunique_count` == 0,0,0.00,[]


In [10]:
df = df.drop('prtt_rate', axis=1)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1208825 entries, 0 to 1208824
Columns: 155 entries, ticker to prdy_vol
dtypes: datetime64[ns](8), float64(63), int64(63), object(21)
memory usage: 1.4+ GB


In [12]:
import pandas as pd
import numpy as np
from scipy.stats import median_abs_deviation
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

def create_description_df(df: pd.DataFrame) -> pd.DataFrame:
  """
  Tier 1(필수)과 Tier 2(가치있는) 통계 지표를 모두 포함하는
  숫자형 데이터 기술 통계 보고서(description_df)를 생성합니다.

  Args:
    df (pd.DataFrame): 분석할 원본 데이터프레임

  Returns:
    pd.DataFrame: 숫자형 피처들의 상세 통계 정보가 담긴 데이터프레임
  """
  # 숫자형 컬럼만 선택
  numeric_df = df.select_dtypes(include=np.number)
  
  # Tier 1: 기본 통계량 + 퍼센타일 (1%, 99% 포함)
  # pandas.describe()를 최대한 활용
  percentiles = [0.01, 0.25, 0.5, 0.75, 0.99]
  desc = numeric_df.describe(percentiles=percentiles).T
  
  # Tier 2: 가치있는 지표들 추가
  
  # 왜도 및 첨도
  desc['skew'] = numeric_df.skew()
  desc['kurtosis'] = numeric_df.kurt()
  
  # IQR (사분위 범위)
  desc['iqr'] = desc['75%'] - desc['25%']
  
  # MAD (중앙값 절대 편차)
  desc['mad'] = median_abs_deviation(numeric_df.dropna())
  
  # CV (변동 계수)
  # 평균이 0일 경우 DivisionByZeroError 방지
  desc['cv'] = desc['std'] / desc['mean']
  desc['cv'] = desc['cv'].replace([np.inf, -np.inf], np.nan)
  
  # 다중공선성 (VIF) - 변수가 2개 이상일 때만 계산
  if numeric_df.shape[1] >= 2:
    # VIF 계산을 위해 상수항 추가
    X = sm.add_constant(numeric_df.dropna())
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    # 상수항(const)의 VIF는 제외하고, 원래 피처 이름에 맞춰 할당
    vif_data = vif_data[vif_data["feature"] != "const"].set_index("feature")
    desc['vif'] = vif_data['VIF']

  # 최대 상관계수
  corr_matrix = numeric_df.corr().abs()
  # 자기 자신과의 상관(1.0)을 제외하기 위해 1.0을 NaN으로 바꿈
  np.fill_diagonal(corr_matrix.values, np.nan)
  desc['max_corr_feature'] = corr_matrix.idxmax()
  desc['max_corr_value'] = corr_matrix.max()

  return desc

# 함수 실행 및 결과 확인
description_df = create_description_df(df)

In [13]:
description_df.T

,prdy_vrss_x,acml_vol_x,acml_tr_pbmn_x,stck_oprc_x,stck_hgpr_x,stck_lwpr_x,stck_clpr,sale_account,sale_cost,sale_totl_prfi,bsop_prti,op_prfi,thtr_ntin,cras,fxas,total_aset,flow_lblt,fix_lblt,total_lblt,total_cptl,grs,bsop_prfi_inrt,ntin_inrt,roe_val,eps_x,sps,bps_x,rsrv_rate,lblt_rate,bram_depn,crnt_rate,quck_rate,equt_inrt,totl_aset_inrt,cptl_ntin_rate,sale_ntin_rate,sale_totl_rate,payout_rate,ebitda,lstg_stqt,cpta,papr,sbst_pric,thdt_clpr,bfdy_clpr,marg_rate,prdy_vrss_y,prdy_ctrt,acml_tr_pbmn_y,acml_vol_y,prdy_vrss_vol_rate,stck_oprc_y,stck_hgpr_y,stck_lwpr_y,stck_mxpr,stck_llam,stck_sdpr,wghn_avrg_stck_prc,hts_frgn_ehrt,frgn_ntby_qty,pgtr_ntby_qty,pvt_scnd_dmrs_prc,pvt_frst_dmrs_prc,pvt_pont_val,pvt_frst_dmsp_prc,pvt_scnd_dmsp_prc,dmrs_val,dmsp_val,stck_sspr,aspr_unit,hts_avls,per,pbr,vol_tnrt,eps_y,bps_y,d250_hgpr,d250_hgpr_vrss_prpr_rate,d250_lwpr,d250_lwpr_vrss_prpr_rate,stck_dryy_hgpr,dryy_hgpr_vrss_prpr_rate,stck_dryy_lwpr,dryy_lwpr_vrss_prpr_rate,w52_hgpr,w52_hgpr_vrss_prpr_ctrt,w52_lwpr,w52_lwpr_vrss_prpr_ctrt,whol_loan_rmnd_rate,frgn_hldn_qty,last_ssts_cntg_qty,aspr_acpt_hour,askp1,askp10,bidp1,bidp10,askp_rsqn1,askp_rsqn2,askp_rsqn3,askp_rsqn4,askp_rsqn5,askp_rsqn6,askp_rsqn7,askp_rsqn8,askp_rsqn9,askp_rsqn10,bidp_rsqn1,bidp_rsqn2,bidp_rsqn3,bidp_rsqn4,bidp_rsqn5,bidp_rsqn6,bidp_rsqn7,bidp_rsqn8,bidp_rsqn9,bidp_rsqn10,total_askp_rsqn,total_bidp_rsqn,ntby_aspr_rsqn,prdy_clpr_vrss_oprc_rate,lwpr_vrss_prpr,prdy_clpr_vrss_lwpr_rate,hgpr_vrss_prpr,prdy_clpr_vrss_hgpr_rate,oprc_vrss_prpr,prdy_vol
count,1108619.0,1192020.0,1192196.0,1208735.0,1208735.0,1208735.0,1208735.0,1202340.0,1134489.0,1202340.0,1202340.0,1202090.0,1202092.0,1086497.0,1086497.0,1200371.0,1086497.0,1086497.0,1200371.0,1200371.0,1197393.0,1038481.0,978997.0,1198273.0,1202267.0,1202328.0,1200371.0,1196550.0,1200066.0,1196488.0,1086497.0,1086497.0,1193148.0,1194005.0,1198401.0,1202278.0,1202340.0,1051721.0,1202219.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1150281.0,1150281.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1202230.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1187501.0,1208825.0,1208825.0,1208825.0,1187501.0,1208825.0,1208825.0,1208825.0,1187501.0,1208825.0,1208825.0,1202277.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1208825.0,1061776.0,1065475.0,1171595.0,1108292.0,1053522.0,1151054.0,1208825.0
mean,-201.08343,687534.444588,22683411411.041157,160660.094807,163297.717015,158115.514226,160558.111354,91362.070651,72935.566823,22542.437046,7286.967737,7582.019603,6076.892765,91604.152198,165989.888645,411753.751435,69518.693653,69849.748691,279943.376929,131810.39125,10.448453,65.520022,68.793646,8.180898,5475.424417,237032.170803,148995.993754,10201.332788,197.915514,24.440059,178.128983,134.478182,8.366562,7.473045,4.058747,6.087317,27.862061,inf,13176.871188,177809426.420651,474697927500.797119,3073.455959,106430.345675,143903.512034,143903.512034,36.623788,-991.528961,-0.52536,66693579509.0186,737122.232767,147.427878,144484.111286,146351.8204,142298.728439,188191.001113,101469.923984,144847.020752,143973.395434,22.663618,-17227.705953,-15971.904483,149233.04902,147039.529642,144698.22352,142504.704142,140163.39802,147082.451729,142547.626229,107122.884396,177.109433,136531.660168,12.536497,1.796506,0.485159,7652.567237,150413.745305,180859.301539,-17.830785,80849.545272,69.784699,165970.492792,-14.833216,87520.429639,59.600095,186465.439605,-18.787723,82960.774541,67.287296,0.789367,56168389.887423,41673.493813,160000.115867,143991.985908,145757.227477,143733.299001,142043.9805

In [14]:
# 1. 날짜와 종목 순으로 데이터를 정렬
df_sorted = df.sort_values(by=['ticker', 'stck_bsop_date'])

# 2. 그룹화 기준('ticker')과 정렬 기준('stck_bsop_date')을 제외한 모든 컬럼 선택
cols_to_fill = df_sorted.columns.drop(['ticker', 'stck_bsop_date'])

# 3. 선택된 컬럼에 대해서만 그룹별 ffill을 적용
df_sorted[cols_to_fill] = df_sorted.groupby('ticker')[cols_to_fill].ffill()

# 4. 남은 결측치를 0으로 최종 처리
df_filled = df_sorted.fillna(0)

# 5. (검증) 결과 확인
# df_filled.isnull().sum().sum() 을 실행하면 0이 나와야 합니다.
print(f"최종 처리 후 남은 결측치 개수: {df_filled.isnull().sum().sum()}")

info_df_filled = create_data_info_df(df_filled)


최종 처리 후 남은 결측치 개수: 0


In [15]:
info_df_filled
#df_filled.isnull().sum()

,dtype,missing_count,missing_percent,nunique_count,nunique_percent
ticker,object,0,0.0,200,0.02
w52_hgpr_date,datetime64[ns],0,0.0,95,0.01
dryy_hgpr_vrss_prpr_rate,float64,0,0.0,190,0.02
dryy_hgpr_date,datetime64[ns],0,0.0,93,0.01
stck_dryy_lwpr,int64,0,0.0,192,0.02
...,...,...,...,...,...
bfdy_clpr,int64,0,0.0,190,0.02
std_idst_clsf_cd_name,object,0,0.0,67,0.01
idx_bztp_lcls_cd_name,object,0,0.0,5,0.00
idx_bztp_scls_cd_name,object,0,0.0,19,0.00


In [16]:
import pandas as pd
import numpy as np

# 1. 수치형 컬럼 선택
numeric_cols = df_filled.select_dtypes(include=np.number).columns.tolist()
# 2. 범주형 컬럼 선택
categorical_cols = df_filled.select_dtypes(include=['category', 'object', 'bool']).columns.tolist()
# 3. Datetime 컬럼 선택
datetime_cols = df_filled.select_dtypes(include=['datetime64[ns]']).columns.tolist()

# --- 결과 확인 ---
print(f"총 {len(df.columns)}개의 컬럼이 3개의 그룹으로 분리되었습니다.")
print("-" * 50)
print(f"수치형 컬럼 ({len(numeric_cols)}개)")
print(f"범주형 컬럼 ({len(categorical_cols)}개)")
print(f"Datetime 컬럼 ({len(datetime_cols)}개)")
print("-" * 50)

# 최종 검증
total_classified = len(numeric_cols) + len(categorical_cols) + len(datetime_cols)
print(f"검증: {total_classified + 1} (분류된 컬럼) == {len(df.columns)} (전체 컬럼)")

총 155개의 컬럼이 3개의 그룹으로 분리되었습니다.
--------------------------------------------------
수치형 컬럼 (126개)
범주형 컬럼 (21개)
Datetime 컬럼 (8개)
--------------------------------------------------
검증: 156 (분류된 컬럼) == 155 (전체 컬럼)


In [17]:
info_df_categorical = create_data_info_df(df[categorical_cols])

In [18]:
info_df_categorical.sort_values(by="nunique_count")

,dtype,missing_count,missing_percent,nunique_count,nunique_percent
hgpr_vrss_prpr_sign,object,0,0.00,2,0.00
lwpr_vrss_prpr_sign,object,0,0.00,2,0.00
new_mkop_cls_code,object,0,0.00,2,0.00
iscd_stat_cls_code,object,0,0.00,2,0.00
new_hgpr_lwpr_cls_code,object,0,0.00,2,0.00
oprc_vrss_prpr_sign,object,0,0.00,3,0.00
prdy_vrss_sign_y,object,0,0.00,3,0.00
idx_bztp_lcls_cd_name,object,86443,7.15,4,0.00
prdy_vrss_sign_x,object,2196,0.18,5,0.00
flng_cls_code,object,0,0.00,6,0.00


In [19]:
# 제거할 컬럼 목록
drop_cols = [
  'stck_shrn_iscd', 
  'issu_istt_cd', 
  'std_pdno', 
  'prdt_abrv_name', 
  'prdt_name',
  'bstp_kor_isnm', 
  'idx_bztp_scls_cd_name', 
  'idx_bztp_lcls_cd_name',
  'bstp_cls_code',
  'prdy_vrss_sign_y',
  'cpfn_cnnm'
]

In [20]:
df_processed = df_filled.drop(columns=drop_cols)

In [21]:
df_processed

,ticker,stck_bsop_date,prdy_vrss_x,prdy_vrss_sign_x,acml_vol_x,acml_tr_pbmn_x,stck_oprc_x,stck_hgpr_x,stck_lwpr_x,stck_clpr,flng_cls_code,sale_account,sale_cost,sale_totl_prfi,bsop_prti,op_prfi,thtr_ntin,cras,fxas,total_aset,flow_lblt,fix_lblt,total_lblt,total_cptl,grs,bsop_prfi_inrt,ntin_inrt,roe_val,eps_x,sps,bps_x,rsrv_rate,lblt_rate,bram_depn,crnt_rate,quck_rate,equt_inrt,totl_aset_inrt,cptl_ntin_rate,sale_ntin_rate,sale_totl_rate,payout_rate,ebitda,lstg_stqt,cpta,papr,scts_mket_lstg_dt,sbst_pric,thdt_clpr,bfdy_clpr,std_idst_clsf_cd_name,iscd_stat_cls_code,marg_rate,new_hgpr_lwpr_cls_code,prdy_vrss_y,prdy_ctrt,acml_tr_pbmn_y,acml_vol_y,prdy_vrss_vol_rate,stck_oprc_y,stck_hgpr_y,stck_lwpr_y,stck_mxpr,stck_llam,stck_sdpr,wghn_avrg_stck_prc,hts_frgn_ehrt,frgn_ntby_qty,pgtr_ntby_qty,pvt_scnd_dmrs_prc,pvt_frst_dmrs_prc,pvt_pont_val,pvt_frst_dmsp_prc,pvt_scnd_dmsp_prc,dmrs_val,dmsp_val,stck_sspr,aspr_unit,hts_avls,per,pbr,vol_tnrt,eps_y,bps_y,d250_hgpr,d250_hgpr_date,d250_hgpr_vrss_prpr_rate,d250_lwpr,d250_lwpr_date,d250_lwpr_vrss_prpr_rate,stck_dryy_hgpr,dryy_hgpr_vrss_prpr_rate,dryy_hgpr_date,stck_dryy_lwpr,dryy_lwpr_vrss_prpr_rate,dryy_lwpr_date,w52_hgpr,w52_hgpr_vrss_prpr_ctrt,w52_hgpr_date,w52_lwpr,w52_lwpr_vrss_prpr_ctrt,w52_lwpr_date,whol_loan_rmnd_rate,frgn_hldn_qty,last_ssts_cntg_qty,aspr_acpt_hour,askp1,askp10,bidp1,bidp10,askp_rsqn1,askp_rsqn2,askp_rsqn3,askp_rsqn4,askp_rsqn5,askp_rsqn6,askp_rsqn7,askp_rsqn8,askp_rsqn9,askp_rsqn10,bidp_rsqn1,bidp_rsqn2,bidp_rsqn3,bidp_rsqn4,bidp_rsqn5,bidp_rsqn6,bidp_rsqn7,bidp_rsqn8,bidp_rsqn9,bidp_rsqn10,total_askp_rsqn,total_bidp_rsqn,ntby_aspr_rsqn,new_mkop_cls_code,prdy_clpr_vrss_oprc_rate,lwpr_vrss_prpr,lwpr_vrss_prpr_sign,prdy_clpr_vrss_lwpr_rate,hgpr_vrss_prpr,hgpr_vrss_prpr_sign,prdy_clpr_vrss_hgpr_rate,oprc_vrss_prpr,oprc_vrss_prpr_sign,prdy_vol
0,000080,1985-01-25,0.0,3,0.0,0.000000e+00,6403.0,6403.0,6403.0,6403.0,00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,70133611,363168055000,5000,2009-10-19,15320,19160,19160,알콜음료 제조업,55,40.0,NaN,-20.0,-0.10,3151539425,164607,85.87,19170,19180,19110,24900,13430,19180,19144.18,7.58,-17687.0,-23952,19413,19296,19223,19106,19033,19260,19070,15340,10,13438,14.25,1.13,0.23,1345.0,16889,22200,2025-07-07,-13.69,18680,2025-02-04,2.57,22200,-13.69,2025-07-07,18680,2.57,2025-02-04,22200,-13.69,2025-07-07,18680,2.57,2025-02-04,1.07,5313280,48024,160000,19170,19260,19160,19070,2593,1470,254,181,25,135,3,1174,23,46,298,5086,1207,5708,5010,7074,5596,2932,5652,1119,5904,39682,33778,31,-0.05,50.0,2,-0.36,-20.0,5,0.00,-10.0,5,191693
1,000080,1985-01-26,0.0,3,0.0,0.000000e+00,6403.0,6403.0,6403.0,6403.0,00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,70133611,363168055000,5000,2009-10-19,15320,19160,19160,알콜음료 제조업,55,40.0,NaN,-20.0,-0.10,3151539425,164607,85.87,19170,19180,19110,24900,13430,19180,19144.18,7.58,-17687.0,-23952,19413,19296,19223,19106,19033,19260,19070,15340,10,13438,14.25,1.13,0.23,1345.0,16889,22200,2025-07-07,-13.69,18680,2025-02-04,2.57,22200,-13.69,2025-07-07,18680,2.57,2025-02-04,22200,-13.69,2025-07-07,18680,2.57,2025-02-04,1.07,5313280,48024,160000,19170,19260,19160,19070,2593,1470,254,181,25,135,3,1174,23,46,298,5086,1207,5708,5010,7074,5596,2932,5652,1119,5904,39682,33778,31,-0.05,50.0,2,-0.36,-20.0,5,0.00,-10.0,5,191693
2,000080,1985-01-28,0.0,3,0.0,0.000000e+00,6403.0,6403.0,6403.0,6403.0,00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,70133611,363168055000,5000,2009-10-19,15320,19160,19160,알콜음료 제조업,55,40.0,NaN,-20.0,-0.10,3151539425,164607,85.87,19170,19180,19110,24900,13430,19180,19144.18,7.58,-17687.0,-23952,19413,19296,19223,19106,19033,19260,19070,15340,10,13438,14.25,1.13,0.23,1345.0,16889,22200,2025-07-07,-13.69,18680,2025-02-04,2.57,22200,-13.69,202

In [23]:
data_store = DBHandler(db_name="data_store")
data_store.replace_table_from_df(df=df_processed, table_name='kr_stock_data')

1. 데이터 준비 
  - 주가 예측에 사용할 금융 피쳐를 수집하고 정제 및 전처리
2. 시장 국면 정의
  - AutoEncoder를 활용하여 전체 기간을 N개의 시장 국면으로 자동 분류한다.
3. 국면별 예측 모델 학습
  - 국면 라벨을 기준으로 데이터를 '상승장 데이터'와 '하락장 데이터'로 분리
  - 각 데이터셋에 대해 개별적인 예측 모델을 따로 학습
4. 국면별 XAI 비교 분석
  - '상승장 모델'과 '하락장 모델'에 각각 SHAP, statsmodel, scikit-learn을 통한 피쳐 중요도를 추출하고, 두 결과를 비교 분석한다.
